Embedding the "places.csv" file (each place gets a vector using sentence-transformers library)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load data
data = pd.read_csv('C:/Users/aarit/OneDrive/Documents/GitHub/DSCDatathon/datatesting/files/p2/datathon_p2/places.csv')

# Preprocessing: Combine columns to form one string per location
def preprocess_row(row):
    tags = row['tags'].strip('{}').replace('_', ' ').replace(',', ' ')
    neighborhood = row['neighborhood'] if pd.notnull(row['neighborhood']) else ''
    emoji = row['emoji'] if pd.notnull(row['emoji']) else ''
    text = f"{row['name']} {row['short_description']} {tags} {neighborhood} {emoji}"
    return text.strip()


data['preprocessed_text'] = data.apply(preprocess_row, axis=1)

# Initialize model, the model being used is a DEFAULT (this model can be changed)
# Other models may provide better semantic/contextual meaning to each sentence
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed the preprocessed text
embeddings = model.encode(data['preprocessed_text'].tolist())

# Optionally: Save the embeddings alongside original data
import numpy as np
embedding_df = pd.DataFrame(embeddings)
data_with_embeddings = pd.concat([data, embedding_df], axis=1)
data_with_embeddings.to_csv('places_with_embeddings.csv', index=False)

# Now you have embeddings that you can use for similarity search


C:\Users\aarit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aarit\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Stora